In [1]:
from data_import import *
from data_preprocess import *

In [2]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.append('../NLP Topic Recognition')
from topic_recognition import *
#from NLP_class import *

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [3]:
from tensorflow import keras
model = keras.models.load_model('../NLP Model Training/model_100K_glove')

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 100)           10697600  
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 10,778,101
Trainable params: 80,501
Non-trainable params: 10,697,600
_________________________________________________________________


In [7]:
txt_folder = Path('../Data/textual_raw').rglob('*.csv')
paths = [x for x in txt_folder]
i = 1
df = pd.DataFrame()
#btc_data = preprocess_BTC()
# _, flagged_usernames = flag_users()
flagged_usernames = []
start = timer()
for path in paths:
#     if i<20:
#         i+=1
#         continue
    message = "Iterration " + str(i) + " of " + str(len(paths))
    print(message)
    text_data = preprocess_textual(path)
    
#     old
#     timetable = topic_recognition(text_data, flagged_usernames = [], display = message)
#     print(timetable)

    text = NLP_preprocess(text_data)
    text.preprocess_data()
    text.import_tokenizer()
    df_padded = text.tokenize_and_pad(train = False)
    
    text_data["sentiment"] = (model.predict(df_padded))
    #print(text_data)
    text_data["hour"] =  text_data.date.map(lambda x: x[:-6])
    timetable = pd.DataFrame(text_data.groupby(['hour']).sentiment.apply(np.mean).values, columns = ["Bitcoin"])
    timetable2 = pd.DataFrame(text_data.groupby(['hour']).sentiment.apply(np.median).values, columns = ["Bitcoin_median"])
    
    timetable = timetable[:-1]
    timetable2 = timetable2[:-1]
    
    btc_connected = connect_datasets(text_data, btc_data)[1:]
    
    i += 1
    
    if len(timetable.index) != len(btc_connected.index):
        continue
        
    timetable.index = btc_connected.index
    timetable2.index = btc_connected.index
    
    timetable = pd.concat([timetable['Bitcoin'], timetable2['Bitcoin_median'], btc_connected['Open']], axis = 1)
    #print(timetable)
    df = pd.concat([df, timetable], axis = 0)

df.to_csv("Dataset_sentiment_100K_glove.csv")
print(timer() - start)


Iterration 1 of 65
Iterration 2 of 65
Iterration 3 of 65
Iterration 4 of 65
Iterration 5 of 65
Iterration 6 of 65
Iterration 7 of 65
Iterration 8 of 65
Iterration 9 of 65
Iterration 10 of 65
Iterration 11 of 65
Iterration 12 of 65
Iterration 13 of 65
Iterration 14 of 65
Iterration 15 of 65
Iterration 16 of 65
Iterration 17 of 65
Iterration 18 of 65
Iterration 19 of 65
Iterration 20 of 65
Iterration 21 of 65
Iterration 22 of 65
Iterration 23 of 65
Iterration 24 of 65
Iterration 25 of 65
Iterration 26 of 65
Iterration 27 of 65
Iterration 28 of 65
Iterration 29 of 65
Iterration 30 of 65
Iterration 31 of 65
Iterration 32 of 65
Iterration 33 of 65
Iterration 34 of 65
Iterration 35 of 65
Iterration 36 of 65
Iterration 37 of 65
Iterration 38 of 65
Iterration 39 of 65
Iterration 40 of 65
Iterration 41 of 65
Iterration 42 of 65
Iterration 43 of 65
Iterration 44 of 65
Iterration 45 of 65
Iterration 46 of 65
Iterration 47 of 65
Iterration 48 of 65
Iterration 49 of 65
Iterration 50 of 65
Iterratio

In [8]:
df

,Bitcoin,Bitcoin_median,Open
2017-09-13 01,0.619920,0.696973,4128.005439
2017-09-13 02,0.622358,0.712012,4066.801167
2017-09-13 03,0.606128,0.668722,3999.225167
2017-09-13 04,0.588174,0.661256,3959.409667
2017-09-13 05,0.624468,0.733002,3995.046667
...,...,...,...
2019-01-22 18,0.640605,0.700322,3584.926897
2019-01-22 19,0.637766,0.700322,3585.858545
2019-01-22 20,0.632203,0.700322,3585.295893
2019-01-22 21,0.644075,0.708547,3579.910755


In [ ]:
import random
i = random.randint(0,len(text_data.index))
print(text_data.loc[i]['text'])
print(text_data.loc[i]['sentiment'])

In [ ]:
#BTC price data of the next hour standarized
Y = pd.DataFrame(df['Open']).astype(float)
#Y = dataset_standarization(Y)

#Textual data ('Bitcoin' topic popularity) of the last hour standarized
X = pd.DataFrame(df['Bitcoin']).astype(float)
#X = dataset_standarization(X)
Y.index = X.index

X = sm.add_constant(X)

model = sm.OLS(Y, X, missing = 'drop').fit()

In [ ]:
model.summary()